# Lesson 3: Reflection and Blogpost Writing

## Setup

In [1]:
llm_config = {"model": "gpt-3.5-turbo"}

## The task!

In [2]:
task = '''
        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       '''


## Create a writer agent

In [3]:
import autogen

writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise " 
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

In [4]:
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])

In [5]:
print(reply)

Title: "Unveiling the Power of DeepLearning.AI"

Dive into the realm of artificial intelligence with DeepLearning.AI, a renowned platform offering top-tier courses on deep learning. Whether you're a novice or an expert, the comprehensive programs cater to all levels, providing insights into neural networks, computer vision, NLP, and more. Backed by industry expert Andrew Ng, the courses blend theory with real-world applications, making learning not only informative but also engaging. Step into the future of technology, upgrade your skills, and unlock new opportunities with DeepLearning.AI. Start your AI journey today!


## Adding reflection 

Create a critic agent to reflect on the work of the writer agent.

In [6]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

In [7]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: "Unveiling the Power of DeepLearning.AI"

Dive into the realm of artificial intelligence with DeepLearning.AI, a renowned platform offering top-tier courses on deep learning. Whether you're a novice or an expert, the comprehensive programs cater to all levels, providing insights into neural networks, computer vision, NLP, and more. Backed by industry expert Andrew Ng, the courses blend theory with real-world applications, making learning not only informative but also engaging. Step into the future of technology, upgrade your skills, and unlock new opportunities with DeepLearning.AI. Start your AI journey today!

--------------------------------------------------------------------------------
Critic (to Writer):

Your blog p

## Nested chat

In [8]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)


In [9]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [10]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

In [11]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

## Orchestrate the nested chats to solve the task

In [12]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 1},
]


In [13]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

**Note**: You might get a slightly different response than what's shown in the video. Feel free to try different task.

In [14]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: "Unveiling the Power of DeepLearning.AI"

Dive into the realm of artificial intelligence with DeepLearning.AI, a renowned platform offering top-tier courses on deep learning. Whether you're a novice or an expert, the comprehensive programs cater to all levels, providing insights into neural networks, computer vision, NLP, and more. Backed by industry expert Andrew Ng, the courses blend theory with real-world applications, making learning not only informative but also engaging. Step into the future of technology, upgrade your skills, and unlock new opportunities with DeepLearning.AI. Start your AI journey today!

--------------------------------------------------------------------------------

*******************************


--------------------------------------------------------------------------------
Meta Reviewer (to Critic):

After aggregating the feedback from multiple reviewers, here are the key suggestions for enhancing the writing:

1. **SEO Optimization:**
   - Incorporate keywords like "artificial intelligence courses" and "online deep learning programs" in the title to improve search visibility.
   - Include internal links to relevant pages on the DeepLearning.AI platform for better site structure and user engagement.
   - Add a call-to-action to encourage readers to enroll or explore courses for improved conversion rates and user experience.

2. **Legal Considerations:**
   - Ensure claims about DeepLearning.AI offering "top-tier courses" or being a "renowned platform" are supported by credible sources.
   - Avoid using terms like "step into the future of technology" unless there is concrete evidence to back such claims to prevent potential deceptive advertising issues.
   - Confirm that the

## Get the summary

In [15]:
print(res.summary)

Title: "Empower Your Future with DeepLearning.AI's AI Courses"

Embark on your AI journey with DeepLearning.AI's cutting-edge artificial intelligence courses and online deep learning programs. Led by industry expert Andrew Ng, these comprehensive offerings cater to learners of all levels, delivering practical insights into neural networks, computer vision, NLP, and more. Elevate your skills, stay ahead in the tech landscape, and explore new opportunities. Enhance your career today by enrolling in DeepLearning.AI's acclaimed courses. Take the first step towards a brighter future in AI!
